# 4.2 LoRAによる質問応答のファインチューニング（推論コード）

本Notebookでは、「2-2_LoRA.ipynb」でファインチューニングした文章生成AIモデルの評価を行います。  
環境については、学習コードで使用した同じ環境を利用します。

## 評価

### (1) ファインチューニング済みモデルの読み込み

学習コードでファインチューニングしたモデルを読み込みます。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

import warnings
warnings.simplefilter("ignore")


# トークナイザ・モデルを読み込みます
tokenizer = AutoTokenizer.from_pretrained("llm-jp/llm-jp-13b-v1.0", use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    "llm-jp/llm-jp-13b-v1.0",
    load_in_8bit=True, # 計算を効率化させるため、重みをint8として読み込みます
    device_map="auto"
  )

peft_model = PeftModel.from_pretrained(
    model,
    "./training_logs"
)
peft_model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50688, 5120)
        (wpe): Embedding(2048, 5120)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-39): 40 x GPT2Block(
            (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
             

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset


# JCommonsenseQAをダウンロードし、データフレームに変換する
dataset_qa = load_dataset("shunk031/JGLUE", name="JCommonsenseQA")
train_df_qa = dataset_qa["train"].to_pandas()
test_df_qa = dataset_qa["validation"].to_pandas()

# JCommonsenseQAデータセットの整備
def preprocess_qa_df(df):
  qa_instruction_text = "質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。"
  df["instruction"] = qa_instruction_text
  df["input"] = df.apply(lambda x: f"質問:\n{x['question']}\n選択肢:\n0.{x['choice0']} 1.{x['choice1']} 2.{x['choice2']} 3.{x['choice3']} 4.{x['choice4']}", axis=1)
  df["output"] = df["label"].astype(str)
  df = df[["instruction", "input", "output"]]
  return df

train_df_qa = preprocess_qa_df(train_df_qa)
test_df_qa = preprocess_qa_df(test_df_qa)

# datasetクラスに変換する
dataset_train = Dataset.from_pandas(train_df_qa)
dataset_train = dataset_train.train_test_split(train_size=0.9, seed=42)
dataset_test = Dataset.from_pandas(test_df_qa)

In [ ]:
# 推論用にデータセット前処理
def add_text(example):
    example["text"] = f"### 指示:\n{example['instruction']}\n\n### 入力:\n{example['input']}\n\n### 回答:\n"
    return example

# 推論パイプライン
def generate(prompt, return_full_text=False, max_token=100):
    input_ids = tokenizer.encode(
        prompt,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(device=peft_model.device)
    output_ids = peft_model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.pad_token_id,
        max_length=max_token,
        temperature=0,
        num_return_sequences=1
    )
    if return_full_text:
        return tokenizer.decode(output_ids.tolist()[0])
    output = tokenizer.decode(output_ids.tolist()[0][input_ids.size(1):])
    return output

dataset_test = dataset_test.map(add_text)

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

### (2) 精度評価

In [ ]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score


def predict(prompt):
    answer = generate(prompt, False)
    try:
      return int(answer[0])
    except ValueError:
      return -1

In [ ]:
y_preds = []
for i in tqdm(range(len(dataset_test))):
    y_preds.append(predict(dataset_test["text"][i]))

y_true = dataset_test["output"]
y_true = [int(i) for i in y_true]

y_preds = np.array(y_preds)
y_true = np.array(y_true)

print(f"accuracy : {accuracy_score(y_true, y_preds):.3f}")

100%|██████████| 1119/1119 [04:54<00:00,  3.80it/s]

accuracy : 0.868


## 応用レシピ

### (1) 類似文章生成

学習したモデルに対して、学習したデータと類似する文章を生成させます。

In [ ]:
prompt = """
### 指示:
質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。

### 入力:
質問:
日本一高い山は？
選択肢:
"""

print(generate(prompt, True, 256))


### 指示:
質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。

### 入力:
質問:
日本一高い山は？
選択肢:
0.富士山 1.東京タワー 2.京都タワー 3.大阪城 4.スカイツリー

### 回答:
0<EOD|LLM-jp>


In [ ]:
prompt = """
### 指示:
質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。

### 入力:
質問:
パソコンを操作するときに扱うものは？
選択肢:
"""

print(generate(prompt, True, 256))


### 指示:
質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。

### 入力:
質問:
パソコンを操作するときに扱うものは？
選択肢:
0.マウス 1.キーボード 2.マウスパッド 3.マウスウォッシュ 4.マウスコンピューター

### 回答:
1<EOD|LLM-jp>


In [ ]:
prompt = """
### 指示:
質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。

### 入力:
質問:
夏の七草といえば？
選択肢:
"""

print(generate(prompt, True, 256))


### 指示:
質問と選択肢を入力として、選択肢から回答を出力してください。また、回答は選択肢から１つを選択し、番号で回答してください。数値で回答し、他の文字は含めないでください。

### 入力:
質問:
夏の七草といえば？
選択肢:
0.すずしろ 1.すずめ 2.すずめの涙 3.すずめの巣 4.すずめの涙の涙

### 回答:
0<EOD|LLM-jp>
